# object projection using depth map

In [2]:
import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [3]:
from scipy.ndimage import gaussian_filter

def smooth_coordinates(new_xs, new_ys, sigma=1):
    """
    좌표값에 가우시안 필터를 적용하여 스무딩.
    """
    smoothed_xs = gaussian_filter(new_xs.astype(float), sigma=sigma)
    smoothed_ys = gaussian_filter(new_ys.astype(float), sigma=sigma)
    return smoothed_xs.astype(int), smoothed_ys.astype(int)


In [4]:
def apply_median_filter(frame):
    """
    색 채널이 (0, 0, 0)인 부분에 평균 필터를 적용합니다.
    """
    # 검은색 픽셀을 마스크로 생성 (RGB가 모두 0인 경우)
    mask = np.all(frame == [0, 0, 0], axis=-1)

    # 마스크 영역을 제외한 부분에 평균 필터 적용
    filtered_frame = cv2.medianBlur(frame, 7)

    # 원본 프레임의 마스크된 부분은 그대로 유지하고, 나머지 부분에 필터링된 결과를 적용
    output_frame = np.copy(frame)
    output_frame[mask] = filtered_frame[mask]

    return output_frame


In [5]:
def orthographic_projection(input_video_path, depth_map_tensor, mask_tensor, output_video_path, focal_length):
    cap = cv2.VideoCapture(input_video_path)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    # 첫 번째 프레임 처리
    ret, frame = cap.read()
    if not ret:
        return

    # 마스크를 적용하여 깊이 맵 추출
    masked_depth = depth_map_tensor * mask_tensor

    # 마스크된 픽셀 인덱스 가져오기
    mask_indices = np.argwhere(mask_tensor.numpy())
    depth_values = masked_depth[mask_indices[:, 0], mask_indices[:, 1]].cpu().numpy()  # Ensure to move to CPU

    # 깊이 값의 최소, 최대 범위 정의 (0으로 나누는 경우 방지)
    min_depth = np.min(depth_values)
    max_depth = np.max(depth_values)

    # 깊이에 따른 스케일링 적용 (깊이에 반비례하여 적용)
    scale_factor = (max_depth - depth_values) / (max_depth - min_depth + 1e-6)

    # z축 정사영 변환 적용
    new_xs = (mask_indices[:, 1] + (depth_values * scale_factor)).astype(int)
    new_ys = (mask_indices[:, 0] + (depth_values * scale_factor)).astype(int)

    # # 좌표 스무딩 적용
    # new_xs, new_ys = smooth_coordinates(new_xs, new_ys, sigma=1)

    # 유효한 좌표 필터링
    valid_mask = (0 <= new_xs) & (new_xs < width) & (0 <= new_ys) & (new_ys < height)

    # 비디오 프레임에 새로운 픽셀 값 할당
    if np.any(valid_mask):  # valid_mask가 True인 경우만
        frame[new_ys[valid_mask], new_xs[valid_mask]] = frame[mask_indices[valid_mask][:, 0], mask_indices[valid_mask][:, 1]]
        
    # 정사영된 좌표 외의 영역을 검은색으로 설정
    valid_projected_mask = np.zeros((height, width), dtype=bool)
    valid_projected_mask[new_ys[valid_mask], new_xs[valid_mask]] = True
    frame[~valid_projected_mask] = [0, 0, 0]  # 정사영된 좌표 외부를 검은색으로 설정
    
    smoothed_frame = apply_median_filter(frame)

    out.write(smoothed_frame)

    # 나머지 프레임 처리
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # 깊이에 따른 스케일링 적용
        scale_factor = (max_depth - depth_values) / (max_depth - min_depth + 1e-6)

        # z축 정사영 변환 적용
        new_xs = (mask_indices[:, 1] + (depth_values * scale_factor)).astype(int)
        new_ys = (mask_indices[:, 0] + (depth_values * scale_factor)).astype(int)

        # # 좌표 스무딩 적용
        # new_xs, new_ys = smooth_coordinates(new_xs, new_ys, sigma=1)

        # 유효한 좌표 필터링
        valid_mask = (0 <= new_xs) & (new_xs < width) & (0 <= new_ys) & (new_ys < height)
        if np.any(valid_mask):  # valid_mask가 True인 경우만
            frame[new_ys[valid_mask], new_xs[valid_mask]] = frame[mask_indices[valid_mask][:, 0], mask_indices[valid_mask][:, 1]]

        # 정사영된 좌표 외의 영역을 검은색으로 설정
        valid_projected_mask = np.zeros((height, width), dtype=bool)
        valid_projected_mask[new_ys[valid_mask], new_xs[valid_mask]] = True
        frame[~valid_projected_mask] = [0, 0, 0]  # 정사영된 좌표 외부를 검은색으로 설정

        smoothed_frame = apply_median_filter(frame)
        out.write(smoothed_frame)

    cap.release()
    out.release()
    cv2.destroyAllWindows()


In [8]:
input_video_path = r"C:\Users\dockn\Downloads\나주대교.mp4"    # Path to the 2D input video
output_video_path = "나주대교.mp4"    # Path to save the output video

depth_map_tensor = torch.load(r"C:\Users\dockn\SAM-code\나주대교_depth.pt")
focal_length = torch.load(r"C:\Users\dockn\SAM-code\나주대교_focal.pt")

mask = plt.imread(r"C:\Users\dockn\OneDrive\바탕 화면\나주.png")
binary = np.logical_not(mask[:,:,-1] == 0)
mask_tensor = torch.from_numpy(binary)

orthographic_projection(input_video_path, depth_map_tensor, mask_tensor, output_video_path, focal_length)

C:\Users\dockn\AppData\Local\Temp\ipykernel_40976\1442653228.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  depth_map_tensor = torch.load(r"C:\Users\dockn\SAM-code\나주대교

In [13]:
input_video_path = r"C:\Users\dockn\Downloads\test.mp4"  # Path to the 2D input video
output_video_path = "삼례_projection_z.mp4"  # Path to save the output video

depth_map_tensor = torch.load(r"C:\Users\dockn\SAM-code\삼례_depth.pt")
mask = plt.imread(r"C:\Users\dockn\OneDrive\바탕 화면\삼례_mask.png")
binary = np.logical_not(mask[:, :, -1] == 0)
mask_tensor = torch.from_numpy(binary)

# 사용자가 알고 있는 초점거리
focal_length = torch.load(r'C:\Users\dockn\SAM-code\삼례_focal.pt')

orthographic_projection(input_video_path, depth_map_tensor, mask_tensor, output_video_path, focal_length)


C:\Users\dockn\AppData\Local\Temp\ipykernel_13688\1207220125.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  depth_map_tensor = torch.load(r"C:\Users\dockn\SAM-code\삼례_d

In [1]:
import cv2
import numpy as np

def orthographic_projection_image(input_image_path, depth_map_tensor, mask_tensor, output_image_path, focal_length):
    # 이미지 읽기
    frame = cv2.imread(input_image_path)

    height, width, _ = frame.shape

    # 마스크를 적용하여 깊이 맵 추출
    masked_depth = depth_map_tensor * mask_tensor

    # 마스크된 픽셀 인덱스 가져오기
    mask_indices = np.argwhere(mask_tensor.numpy())
    depth_values = masked_depth[mask_indices[:, 0], mask_indices[:, 1]].cpu().numpy()  # Ensure to move to CPU

    # 깊이 값의 최소, 최대 범위 정의 (0으로 나누는 경우 방지)
    min_depth = np.min(depth_values)
    max_depth = np.max(depth_values)

    # 깊이에 따른 스케일링 적용 (깊이에 반비례하여 적용)
    scale_factor = (max_depth - depth_values) / (max_depth - min_depth + 1e-6)

    # z축 정사영 변환 적용
    new_xs = (mask_indices[:, 1] + (depth_values * scale_factor)).astype(int)
    new_ys = (mask_indices[:, 0] + (depth_values * scale_factor)).astype(int)

    # 유효한 좌표 필터링
    valid_mask = (0 <= new_xs) & (new_xs < width) & (0 <= new_ys) & (new_ys < height)

    # 이미지에 새로운 픽셀 값 할당
    if np.any(valid_mask):  # valid_mask가 True인 경우만
        frame[new_ys[valid_mask], new_xs[valid_mask]] = frame[mask_indices[valid_mask][:, 0], mask_indices[valid_mask][:, 1]]

    # 정사영된 좌표 외의 영역을 검은색으로 설정
    valid_projected_mask = np.zeros((height, width), dtype=bool)
    valid_projected_mask[new_ys[valid_mask], new_xs[valid_mask]] = True
    frame[~valid_projected_mask] = [0, 0, 0]  # 정사영된 좌표 외부를 검은색으로 설정

    # 중앙값 필터 적용
    smoothed_frame = apply_median_filter(frame)

    # 처리된 이미지 저장
    cv2.imwrite(output_image_path, smoothed_frame)

def apply_median_filter(image):
    # 중앙값 블러 필터 적용
    return cv2.medianBlur(image, 5)

# 사용 예시
input_image_path = 'input_image.jpg'
output_image_path = 'output_image.jpg'
# depth_map_tensor, mask_tensor는 미리 정의되어야 합니다.
# focal_length는 초점 거리를 사용자가 설정합니다.

# 함수 호출
orthographic_projection_image(input_image_path, depth_map_tensor, mask_tensor, output_image_path, focal_length)
